In [193]:
import os
import csv
import pyarrow.parquet as pq

import pandas as pd
import numpy as np

In [194]:

# data_list_parquet = []

# # Loop through files in directory
# for file in os.listdir(path_data):
#     # Check if file is a Parquet file
#     if file.endswith(".parquet"):
#         # Open Parquet file
#         file_path = os.path.join(path_data, file)
#         table = pq.read_table(file_path)
        
#         # Convert Parquet table to list of rows
#         parquet_table = table.to_pylist()
#         parquet_data = [list(item.values()) for item in parquet_table]
        
#         # Append rows to data_list
#         data_list_parquet.append(parquet_data)

# data_parquet = pd.concat([pd.DataFrame(data) for data in data_list_parquet]).reset_index(drop=True)
# data_parquet.columns = table.schema.names
# data_parquet.head()

In [195]:

# Define path to data
path_data = "../data/Scraped data/May 2023"
brand = "audi"

# empty list to store data
data_list = []

# loop through files in directory
for file in os.listdir(path_data):
    # check if file is in list of files to load
    # if ".csv" in file and brand in file:
    if ".csv" in file:
        # open file and read data
        with open(os.path.join(path_data, file), newline='') as f:
            # create csv reader object
            reader = csv.reader(f)
            # iterate over rows in the csv file and add filename to each row
            reader_data = [row + [file.split(".")[0]] for row in reader]
            # store column names and data in data_list
            col_names = reader_data[0]
            data_list.append(reader_data[1:])

# concatenate data from all files into one dataframe
data = pd.concat([pd.DataFrame(data) for data in data_list]).reset_index(drop=True)

# Drop the first (index as a column) and last column (postcode - the above code must
# have added it)
data.drop(columns=[0, 11], inplace=True)
data.columns = col_names[1:11]

print(data.shape)

# Drop duplicates
data = data.drop_duplicates(subset="carId").reset_index(drop=True)

print(data.shape)

# print first five rows of data
data.head()


(8671, 10)
(5061, 10)


,model,year,price,transmission,mileage,fuelType,mpg,engineSize,brand,carId
0,TT,2019,20999,Automatic,47000,Petrol,40.4,2.0,audi,30029005
1,A4,2003,1995,Automatic,118000,Petrol,33.6,1.8,audi,30040039
2,Q7,2007,6995,Automatic,120252,Diesel,None,3.0,audi,28464166
3,Q7,2007,5950,Automatic,121654,Diesel,26.9,3.0,audi,29973124
4,Q7,2007,5950,Automatic,121654,Diesel,None,3.0,audi,28472877


In [196]:
# Remove cars of fuel type equal to 'Petrol/Electric'
data = data[data["fuelType"] != "Petrol/Electric"]
data.reset_index(drop=True, inplace=True)

# Create a boolean ev_mask for elements containing the word "Electric"
ev_mask = data.apply(lambda x: x.str.contains('Electric', case=False)).any(axis=1)

ev_mileage = data.loc[ev_mask, "mpg"]
data = data.copy()
data.loc[ev_mask, "mileage"] = ev_mileage

# Clean data from electric cars. Because some fields such as mpg or engineSize doesn't
# really apply to them, the scraping tool didn't get the information right
data.loc[ev_mask, "transmission"] = "Automatic"
data.loc[ev_mask, "mpg"] = 0
data.loc[ev_mask, "fuelType"] = "Electric"
data.loc[ev_mask, "engineSize"] = 0

data[ev_mask].sample(10)

,model,year,price,transmission,mileage,fuelType,mpg,engineSize,brand,carId
4861,Q4 e-tron,2021,48195,Automatic,9776,Electric,0,0,audi,28702653
1173,e-tron,2021,76890,Automatic,13368,Electric,0,0,audi,29900296
1730,e-tron,2021,73490,Automatic,17999,Electric,0,0,audi,30033830
1259,e-tron,2022,51000,Automatic,10759,Electric,0,0,audi,29995061
3510,e-tron,2022,79999,Automatic,5917,Electric,0,0,audi,29462595
1066,e-tron,2021,54690,Automatic,14829,Electric,0,0,audi,29450554
1220,e-tron,2019,44890,Automatic,18000,Electric,0,0,audi,29371878
512,e-tron,2022,72990,Automatic,13468,Electric,0,0,audi,29282524
1731,e-tron,2021,48990,Automatic,24999,Electric,0,0,audi,30046959
1724,e-tron,2022,45490,Automatic,6999,Electric,0,0,audi,30033793


In [197]:
data["mileage"] = pd.to_numeric(data['mileage'], errors='coerce')
data = data.dropna()
data.reset_index(drop=True, inplace=True)

In [198]:
columns_to_apply = data.columns[data.columns != 'mileage']

# Apply the function to selected columns
none_mask = data[columns_to_apply].apply(lambda x: x.str.contains('None', case=False)).any(axis=1)
int_mask = data['mileage'].apply(lambda x: x.is_integer())
data = data[(~none_mask) & (int_mask)]
data.reset_index(drop=True, inplace=True)

In [199]:
int_cols = ["year", "price", "mileage"]
float_cols = ["mpg", "engineSize"]

data[int_cols] = data[int_cols].astype(int)
data[float_cols] = data[float_cols].astype(float)
data["engineSize"] = round(data["engineSize"],1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4825 entries, 0 to 4824
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         4825 non-null   object 
 1   year          4825 non-null   int64  
 2   price         4825 non-null   int64  
 3   transmission  4825 non-null   object 
 4   mileage       4825 non-null   int64  
 5   fuelType      4825 non-null   object 
 6   mpg           4825 non-null   float64
 7   engineSize    4825 non-null   float64
 8   brand         4825 non-null   object 
 9   carId         4825 non-null   object 
dtypes: float64(2), int64(3), object(5)
memory usage: 377.1+ KB


In [200]:
data.sample(10)

,model,year,price,transmission,mileage,fuelType,mpg,engineSize,brand,carId
31,Q8,2018,41950,Automatic,46252,Diesel,33.6,3.0,audi,30038423
270,A3,2018,14349,Manual,47130,Diesel,70.6,1.6,audi,29969167
2437,A4,2017,27950,Automatic,52000,Petrol,38.7,3.0,audi,29746660
3135,A3,2021,29980,Semi-Auto,8997,Diesel,57.7,2.0,audi,29486250
523,TT,2020,27990,Automatic,36288,Petrol,40.9,2.0,audi,29831678
2772,SQ5,2015,24250,Automatic,54000,Diesel,42.8,3.0,audi,30007165
4157,A4,2023,34500,Semi-Auto,2800,Diesel,54.3,2.0,audi,29509719
4196,A5,2022,28400,Semi-Auto,11507,Petrol,45.6,2.0,audi,29469583
1807,A3,2016,14995,Manual,84609,Petrol,60.1,1.4,audi,29926398
4652,A6,2015,18491,Automatic,47000,Diesel,64.2,2.0,audi,30030189


In [201]:
data["brand"].value_counts()

audi    4825
Name: brand, dtype: int64